In [1]:
from datetime import datetime
from math import exp, pow, sqrt
from time import sleep
import pandas as pd
import plotly.graph_objects as go
from tqdm import tqdm

In [2]:
sz50Opt = pd.read_pickle('sz50OptTotal.pkl')
# sz50Opt.head(n=5)

In [3]:
def findNextMonthOpt(df):
    expireDateList = df['tradingdays_to_expiration'].unique().tolist()
    expireDateList.sort()
    # print(expireDateList)
    # 第二小的日期
    nextMonthDate = expireDateList[1]
    # 选择次月期权
    nextMonthOpt = df.loc[df['tradingdays_to_expiration']==nextMonthDate]
    return nextMonthOpt

In [4]:
def calculate_variance(option):
    '''
    Calculates the variance of the option prices
    params:
        option: dataframe,包含了当天的 近月期权 或者 次月期权 信息
        output: 是否打印中间变量
    return:
        vix: float, 计算出来的当日vix值
    '''
    # 第一步：计算剩余时间
    t = (540*2/(24*60) + int(option['tradingdays_to_expiration'].iloc[0])) / 242
    # 第二步：确定贴现利率
    r = 3 / 100
    # 第三步：提取远期合成期货价格
    synthetic_price = float(option['synthetic_futures_price'].iloc[0])
    # 第四步：选择平值期权
    atm_strike = option.loc[option['is_at_the_money']==True, 'strike_price'].iloc[0]
    # 第五步：获取虚值价格
    strikes = list(option['strike_price'].unique())
    strikes.sort()
    otms = {}
    for strike in strikes:
        call = option.loc[(option['strike_price']==strike) & (option['option_type']=='认购')]
        call_price = call['close'].iloc[0]

        put = option.loc[(option['strike_price']==strike) & (option['option_type']=='认沽')]
        put_price = put['close'].iloc[0]

        if strike > atm_strike:
            otms[strike] = call_price
        elif strike < atm_strike:
            otms[strike] = put_price
        else:
            otms[strike] = (call_price + put_price) / 2
    # 第六步：计算行权间距
    strike_diffs = {}
    max_n = len(strikes)
    for n, strike in enumerate(strikes):
        if n < max_n - 1:
            next_strike = strikes[n + 1]
            strike_diff = next_strike - strike
            strike_diffs[strike] = strike_diff
        else:
            strike_diffs[strike] = strike_diff
    # 第七步：计算方差贡献
    variance_data = {}
    for strike in strikes:
        otm_price = otms[strike]
        strike_diff = strike_diffs[strike]
        variance = (strike_diff / pow(strike, 2)) * exp(r * t) * otm_price
        variance_data[strike] = variance
    result = (2 / t) * sum(list(variance_data.values())) - pow((synthetic_price/atm_strike - 1), 2) / t
    return result,t

In [5]:
def calculate_vix(df):
    # 挑选出每天的近月期权, 为tradingdays_to_expiration最小
    sz50CurrentMonthOpt = df.groupby('datetime', as_index=False).apply(lambda x: x[x['tradingdays_to_expiration'] == x['tradingdays_to_expiration'].min()]).reset_index(drop=True)
    # 挑选出每天的次月期权, temp50Opt中tradingdays_to_expiration最小
    sz50NextMonthOpt = df.groupby('datetime', as_index=False).apply(findNextMonthOpt)
    curSigma = pd.DataFrame(columns=['datetime', 'curSigma', 'curT'])
    print('计算近月期权的sigma')
    for date in tqdm(sz50CurrentMonthOpt['datetime'].unique()):
        temp = sz50CurrentMonthOpt.loc[sz50CurrentMonthOpt['datetime'] == date].reset_index(drop=True)
        sigma, t = calculate_variance(temp)
        curSigma = pd.concat([curSigma, pd.DataFrame({'datetime': [date], 'curSigma': [sigma], 'curT': [t]})], ignore_index=True)
    print('计算次月期权的sigma')
    nextSigma = pd.DataFrame(columns=['datetime', 'nextSigma', 'nextT'])
    for date in tqdm(sz50NextMonthOpt['datetime'].unique()):
        temp = sz50NextMonthOpt.loc[sz50NextMonthOpt['datetime'] == date].reset_index(drop=True)
        sigma, t = calculate_variance(temp)
        nextSigma = pd.concat([nextSigma, pd.DataFrame({'datetime': [date], 'nextSigma': [sigma], 'nextT': [t]})], ignore_index=True)
    # merge
    sigma = pd.merge(curSigma, nextSigma, on='datetime', how='outer')
    print('计算vix指数')
    sigma['tm'] = 22 / 242
    sigma['ty'] = 1
    sigma['temp_1'] = sigma['curT'] * sigma['curSigma'] * (sigma['nextT'] - sigma['tm']) / (sigma['nextT'] - sigma['curT'])
    sigma['temp_2'] = sigma['nextT'] * sigma['nextSigma'] * (sigma['tm'] - sigma['curT']) / (sigma['nextT'] - sigma['curT'])
    sigma['vix'] = ((sigma['temp_1'] + sigma['temp_2']) * sigma['ty'] / sigma['tm']).apply(sqrt) * 100
    print('vix指数计算完成')
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sigma['datetime'], y=sigma['vix'], mode='lines', name='vix'))
    fig.update_layout(title='VIX指数', xaxis_title='日期', yaxis_title='VIX指数')
    fig.show()
    return sigma

# 上证50etf VIX

In [6]:
sz50Vix = calculate_vix(sz50Opt)

计算近月期权的sigma


100%|██████████| 2144/2144 [00:37<00:00, 57.31it/s] 


计算次月期权的sigma


100%|██████████| 2144/2144 [00:22<00:00, 96.14it/s] 


计算vix指数
vix指数计算完成


In [7]:
sz50Vix.to_pickle('sz50Vix.pkl')
sz50Vix.head(n=5)

,datetime,curSigma,curT,nextSigma,nextT,tm,ty,temp_1,temp_2,vix
0,2015-02-09,0.066707,0.114669,0.054398,0.193182,0.090909,1,0.009964,-0.003180,27.317309
1,2015-02-10,0.071371,0.110537,0.056700,0.189050,0.090909,1,0.009861,-0.002680,28.106681
2,2015-02-11,0.059506,0.106405,0.046974,0.184917,0.090909,1,0.007581,-0.001714,25.404207
3,2015-02-12,0.058731,0.102273,0.049439,0.180785,0.090909,1,0.006876,-0.001294,24.780018
4,2015-02-13,0.048540,0.098140,0.042275,0.176653,0.090909,1,0.005202,-0.000688,22.284760
